<a href="https://colab.research.google.com/github/pybuttheavy/Prog2kakushin/blob/main/Work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import re

# "\"入りコマンド文字列を区切る。空きスペースはスキップ。大文字小文字は区別しない
def backslash(strs:str):
  rlist=[]
  strs=strs.lower()
  strs=strs.replace(" ","")
  while True:
    a=strs.find("\\")
    if a==-1:
      break
    elif a==0:
      strs=strs[1:]
      continue
    elif strs[-1]=="\\":
      strs=strs[:-1]
      continue
    else:
      c1=strs[:a]
      rlist.append(c1)
      strs=strs[a+1:]
  rlist.append(strs)
  return rlist

# 入力したコマンドの履歴を表示
def p_past_c():
  print("コマンド履歴表示")
  for c in past_c:
    print(c)

# "help"を入力された時に表示する
def p_help():
  print("コマンド一覧")
  print("help                               このコマンド一覧を表示します")
  print("exit                               アプリを終了します")
  print("ls_c                               入力したコマンドの履歴を表示します")
  print("数値　　　　　　　　　　　　　　   数値分前のコマンドを実行します")
  print("input \A \B                        指定したウェブ上のcsvファイル'A'を読み込み、csvファイル'B'として保存します")
  print("                                   'A'は入力必須　'B'が入力されていない場合は自動設定します")
  print("ls_f                               保存しているcsvファイルリストを表示します")
#  print("check \A                            指定したcsvファイル'A'の詳細を表示します")
  print("rename \A \B                       保存しているcsvファイル'A'の名前をcsvファイル'B'に変更します")
  print("del \A                             指定したcsvファイル'A'を削除します")
  print("copy \A \B                         指定したcsvファイル'A'をコピー、ファイル'B'を作成します")
  print("cal \A \B \C                       指定したcsvファイル'A','B'で演算し、新しいファイル'C'を作成します")
  print("marge_csv \A \B \C                 指定したcsvファイル'A','B'を直接結合し、新しいファイル'C'を作成します")
  print("marge_nd \A \B \C                  指定したcsvファイル'A','B'の数値部分を結合し、新しいファイル'C'を作成します")
  print("make \A                            0からcsvファイル'A'を作成、保存します")
#  print("ramdom \A                          csv1ファイル'A'をランダム値で作成します")
  print("manual                             自分でPythonコードを入力して実行します")


# 自動ファイル名設定時に使用。int数字に対し必要に応じて"0"を付け足したstrを返す
def emfil_0s(a):
  if a<10:
    return "00"+str(a)
  elif a<100:
    return "0"+str(a)
  else:
    return str(a)

# 使えないファイル名を使えるように変換する。末尾が".csv"ならそれはそのまま
# '/'もそのまま
# 第二引数がTrueならば、強制的に末尾が".csv"になる
def fileNchange(name,a):
  if len(name)>=4 and name[-4:]==".csv":
    name=name[:-4]
    a=True
  name=re.sub(r'[\\|:|*|?|.|"|<|>|\|]','-',name)
  if a:
    name+=".csv"
  return name




# ここからプログラム本体

# コマンド入力の履歴を定義
past_c=[]

while True:

  # 最新のcsvファイルリスト入手
  csv_list=glob.glob("**/*.csv",recursive=True)

  # コマンドを入力させる
  com=input("コマンド入力>>")

  # コマンドが数値だった場合、その数値分前のコマンドを実行。不適切な数値ならエラー
  try:
    past_int=int(com)
    if len(past_c)>=past_int and past_int>0:
      past_int*=-1
      com2=past_c[past_int]
      com_list=backslash(com2)
      l_c=len(com_list)
      past_c.append(com2)
      print("実行:"+com2)
    else:
      print("エラー:"str(com)+"個前の履歴は存在しません")
      p_past_c()
  except:
    com_list=backslash(com)
    l_c=len(com_list)
    past_c.append(com)

  if com_list[0]=="help":
    p_help:
    continue

  elif com_list[0]=="exit":
    break

  # コマンド履歴表示
  elif com_list[0]=="ls_c":
    p_past_c()
    continue

  elif com_list[0]=="input":
    # ファイルは999まで　ファイル名自動設定の関係で制限
    if len(csv_list)>=999:
      print("エラー:これ以上csvファイルは保存できません。先に不要なcsvファイルを削除してください")
      continue
    # url / uname 初期化
    url=""
    uname=""
    # 引数が入力されているなら格納
    if l_c>=2:
      url=com_list[1]
    if l_c>=3:
      uname=com_list[2]
    # 第一引数が空白なら警告＋コマンド入力に戻る
    if url=="" or url==".csv":
      print("エラー　第一引数:引数まで正確に入力してください")
      continue
    # url 適切な形に改変
    url=fileNchange(url,False)
    print("第一引数:"+url,end="  / 第二引数:",end="")
    # 第二引数が空白なら自動設定
    if uname=="" or uname==".csv":
      file_int=1
      while emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      uname=emfil_0s(file_int)+".csv"
      print("空白(自動設定):"+uname+".csv")
    # 空白でなければ適切な形に変換
    else:
      uname=fileNchange(uname,True)
      print(uname)
    # ダウンロード
    print("info:ウェブ上のファイル:"+url+" をダウンロード")
    print("     ファイル名:"+uname+"としての保存を試行")
    !wget $url -o $uname

  # csvファイルのリスト表示　サイズ＋ファイル名
  elif com_list[0]=="ls_f":
    for n in csv_list:
      df=pd.read_csv(n)
      num=csv_list.index(n)+1
      shapewrite=str(len(df.index))+"*"+str(len(df.columns))
      shapespace=15-len(shapewrite)
      if shapespace<1:
        shapespace=1
      print(str(num)+" "+shapewrite+" "*shapespace+n)
    continue

  # 第一引数のファイルを第二引数の名前に変更
  elif com_list[0]=="rename":
    # yetf aftf 初期化　引数が入力されているなら格納
    yetf=""
    aftf=""
    if l_c>=2:
      yetf=com_list[1]
    if l_c>=3:
      aftf=com_list[2]
    # 第一引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす
    try:
      file_num=int(yetf)
      if file_num<=len(csv_list) and file_num>0:
        yetf2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      yetf2=fileNchange(yetf,True)
    # 第一引数が空白またはファイルが存在しない場合はエラー　コマンド入力に戻る
    if yetf2=="" or yetf2==".csv":
      print("エラー　第一引数:引数まで正確に入力してください")
      continue
    if yetf2 not in csv_list:
      print("エラー　第一引数:変更するファイルが存在しません")
      continue
    # 第二引数が空白なら自動設定
    aftf=fileNchange(aftf,True)
    if aftf=="" or aftf==".csv":
      file_int=1
      while emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      aftf=emfil_0s(file_int)+".csv"
      print("第二引数　空白(自動設定):"+aftf+".csv")
    # 既に存在するファイル名なら後ろに番号を自動設定で追加
    if aftf in csv_list:
      aftf=aftf[:-4]
      file_int=1
      while aftf+emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      aftf+=emfil_0s(file_int)+".csv"
    # ファイル名変更
    os.rename(yetf2,aftf)
    print("info:ファイル "+yetf+" の名前を "+aftf+" に変更")
    continue
  # 引数のファイルを削除
  elif com_list[0]=="del":
    # delf 初期化　引数が入力されているなら格納
    delf=""
    if l_c>=2:
      delf=com_list[1]
    # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす
    try:
      file_num=int(delf)
      if file_num<=len(csv_list) and file_num>0:
        delf2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      delf2=fileNchange(delf,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if delf2=="" or ".csv":
      print("エラー　引数:引数まで正確に入力してください")
      continue
    if delf2 not in csv_list:
      print("エラー　引数:指定されたファイルが存在しません")
      continue
    # ファイル削除
    os.remove(delf2)
    print("info:ファイル "+delf2+" を削除")
    continue
  # csvファイル作成make
  elif com_list[0]=="make":
    # 作成ファイル名設定
    makef=""
    if l_c>=2:
      makef=com_list[1]
    if makef=="" or makef==".csv":
      file_int=1
      while emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      makef=emfil_0s(file_int)+".csv"
      print("空白(自動設定):"+makef+".csv")
    # 空白でなければ適切な形に変換
    else:
      makef=fileNchange(makef,True)
    # 既に存在するファイル名なら後ろに番号を自動設定で追加
    if makef in csv_list:
      makef=makef[:-4]
      file_int=1
      while makef+emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      makef+=emfil_0s(file_int)+".csv"
        continue
    print("info:ファイル名'"+makef+"'を作成")
    # 行列数設定
    print("info:行列数設定（0以下で中止）")
    while True:
      try:
        makere=int(input("列数:"))
        makegy=int(input("行数:"))
      except:
        print("エラー:数字を入力してください")
    if makere<=0 or makegy<=0:
      continue
    # データ入力
    else:
      stopper=False
      value=0
      makelist=[]
      for i in range(makegy):
        makerow=[]
        for j in range(makere):
          value=input("インデックス"+str(i)+","+str(j)+"の値('stop'で中止):")
          if value=="stop":
            stopper=True
            break
          else:
            try:
              valuen=int(value)
            except:
              valuen=value:
            makerow.append(valuen)
        if stopper:
          break
        else:
          makelist.append(makerow)
      if stopper:
        print("作成中止")
        continue
      # columns有無入力
      ques=input("columnsあり? y/other:")
      if ques !="" and ques[0]=="y":
        try:
          makedf=pd.DataFrame(makelist[1:],columns=makelist[0])
        except:
          print("エラー:データフレーム作成失敗")
          continue
      else:
        try:
          makedf=pd.DataFrame(makelist)
        except:
          print("エラー:データフレーム作成失敗")
          continue
      # index有無入力
      makeind=False
      ques2=input("indexあり？ y/other")
      if ques !="" and ques[0]=="y":
        makeind=True
      else:
        makeind=False
      # csv作成
      try:
        makedf.to_csv(makef,index=makeind)
        print("csvファイル'"+makef+"'を作成しました")
      except:
        print("エラー:csvファイル作成失敗")
      continue
    continue
  # csvファイル作成cal
  elif com_list[0]=="cal":
    cal1f=""
    cal2f=""
    cal3f=""
    if l_c>=3:
      cal1f=com_list[1]
      cal2f=com_list[2]
     # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす cal1f
    try:
      file_num=int(cal1f)
      if file_num<=len(csv_list) and file_num>0:
        cal1f2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      cal1f2=fileNchange(basef,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if cal1f2=="" or cal1f2==".csv":
      print("エラー　第一引数:引数まで正確に入力してください")
      continue
    if cal1f2 not in csv_list:
      print("エラー　第二引数:指定されたファイルが存在しません")
      continue
     # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす cal2f
    try:
      file_num=int(cal2f)
      if file_num<=len(csv_list) and file_num>0:
        cal2f2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      cal2f2=fileNchange(basef,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if cal2f2=="" or cal2f2".csv":
      print("エラー　引数:引数まで正確に入力してください")
      continue
    if cal2f2 not in csv_list:
      print("エラー　引数:指定されたファイルが存在しません")
      continue

    # 作成ファイル名設定
    if l_c>=4:
      cal3f=com_list[3]
    if cal3f=="" or cal3f==".csv":
      file_int=1
      while emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cal3f=emfil_0s(file_int)+".csv"
      print("空白(自動設定):"+cal3f+".csv")
    # 空白でなければ適切な形に変換
    else:
      cal3f=fileNchange(cal3f,True)
    # 既に存在するファイル名なら後ろに番号を自動設定で追加
    if cal3f in csv_list:
      cal3f=cal3f[:-4]
      file_int=1
      while cal3f+emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cal3f+=emfil_0s(file_int)+".csv"
        continue
    # 演算


    continue


# csvファイル作成marge_csv
  elif com_list[0]=="marge_csv":
    cal1f=""
    cal2f=""
    cal3f=""
    if l_c>=3:
      cal1f=com_list[1]
      cal2f=com_list[2]
     # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす cal1f
    try:
      file_num=int(cal1f)
      if file_num<=len(csv_list) and file_num>0:
        cal1f2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      cal1f2=fileNchange(basef,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if cal1f2=="" or cal1f2==".csv":
      print("エラー　第一引数:引数まで正確に入力してください")
      continue
    if cal1f2 not in csv_list:
      print("エラー　第二引数:指定されたファイルが存在しません")
      continue
     # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす cal2f
    try:
      file_num=int(cal2f)
      if file_num<=len(csv_list) and file_num>0:
        cal2f2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      cal2f2=fileNchange(basef,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if cal2f2=="" or cal2f2".csv":
      print("エラー　引数:引数まで正確に入力してください")
      continue
    if cal2f2 not in csv_list:
      print("エラー　引数:指定されたファイルが存在しません")
      continue

    # 作成ファイル名設定
    if l_c>=4:
      cal3f=com_list[3]
    if cal3f=="" or cal3f==".csv":
      file_int=1
      while emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cal3f=emfil_0s(file_int)+".csv"
      print("空白(自動設定):"+cal3f+".csv")
    # 空白でなければ適切な形に変換
    else:
      cal3f=fileNchange(cal3f,True)
    # 既に存在するファイル名なら後ろに番号を自動設定で追加
    if cal3f in csv_list:
      cal3f=cal3f[:-4]
      file_int=1
      while cal3f+emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cal3f+=emfil_0s(file_int)+".csv"
        continue
    # 結合
    df1=pd.DataFrame(cal1f)
    df2=pd.DataFrame(calf2)
    ques=input("結合の向きは縦（列の数は変わらず、行が増える）？ y/other")
    if ques !="" and ques[0]=="y":
      marind=0
    else:
      marind=1
    df3=pd.concat([df1,df2],axis=marind)
    print("結合データフレームの上５行表示")
    print(df3[:5])
    ques=input("index表示？ y/other")
    if ques !="" and ques[0]=="y":
      marind2=True
    else:
      marind2=False
    df3.to_csv(cal3f,index=marind2)
    print("ファイル'"+cal1f+"'とファイル'"+cal2f+"'を結合しました")


# csvファイル作成marge_nd
  elif com_list[0]=="marge_nd":
    cal1f=""
    cal2f=""
    cal3f=""
    if l_c>=3:
      cal1f=com_list[1]
      cal2f=com_list[2]
     # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす cal1f
    try:
      file_num=int(cal1f)
      if file_num<=len(csv_list) and file_num>0:
        cal1f2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      cal1f2=fileNchange(basef,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if cal1f2=="" or cal1f2==".csv":
      print("エラー　第一引数:引数まで正確に入力してください")
      continue
    if cal1f2 not in csv_list:
      print("エラー　第二引数:指定されたファイルが存在しません")
      continue
     # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす cal2f
    try:
      file_num=int(cal2f)
      if file_num<=len(csv_list) and file_num>0:
        cal2f2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      cal2f2=fileNchange(basef,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if cal2f2=="" or cal2f2".csv":
      print("エラー　引数:引数まで正確に入力してください")
      continue
    if cal2f2 not in csv_list:
      print("エラー　引数:指定されたファイルが存在しません")
      continue

    # 作成ファイル名設定
    if l_c>=4:
      cal3f=com_list[3]
    if cal3f=="" or cal3f==".csv":
      file_int=1
      while emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cal3f=emfil_0s(file_int)+".csv"
      print("空白(自動設定):"+cal3f+".csv")
    # 空白でなければ適切な形に変換
    else:
      cal3f=fileNchange(cal3f,True)
    # 既に存在するファイル名なら後ろに番号を自動設定で追加
    if cal3f in csv_list:
      cal3f=cal3f[:-4]
      file_int=1
      while cal3f+emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cal3f+=emfil_0s(file_int)+".csv"
        continue
    # 結合
    df1=pd.DataFrame(cal1f)
    df2=pd.DataFrame(cal2f)
    ques=input("結合の向きは縦（列の数は変わらず、行が増える）？ y/other")
    if ques !="" and ques[0]=="y":
      marind=0
    else:
      marind=1

    nd1=df1.to_numpy()
    nd2=df2.to_numpy()
    nd3=np.concatenate([nd1,nd2],marind)
    print("結合した配列の上５行表示")
    print(nd3[:5])
    np.savetxt(cal3f,nd3)

    print("ファイル'"+cal1f+"'とファイル'"+cal2f+"'を結合しました")

  # ファイルコピー
  elif com_list[0]=="copy":
    if l_c<=1:
      print("エラー  引数:引数を入力してください")
      continue
    cpbf=""
    cpmf=""
    if l_c>=2:
      cpbf=com_list[1]
    if l_c>=3:
      cpmf=com_list[2]
     # 引数がファイル名ではなく数値だった場合、その数値に対応するファイルとみなす
    try:
      file_num=int(cpbf)
      if file_num<=len(csv_list) and file_num>0:
        cpbf2=csv_list[file_num-1]
      else:
        print("エラー:番号"+str(file_num)+" のファイルは存在しません")
        continue
    except:
      cpbf2=fileNchange(cpbf,True)
    # 空白または該当ファイルが存在しないならエラー　コマンド入力に戻る
    if cpbf2=="" or ".csv":
      print("エラー　引数:引数まで正確に入力してください")
      continue
    if cpbf2 not in csv_list:
      print("エラー　引数:指定されたファイルが存在しません")
      continue
      # 第二引数が空白なら自動設定
    cpmf=fileNchange(cpmf,True)
    if cpmf=="" or cpmf==".csv":
      file_int=1
      while emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cpmf=emfil_0s(file_int)+".csv"
      print("第二引数　空白(自動設定):"+cpmf+".csv")
    # 既に存在するファイル名なら後ろに番号を自動設定で追加
    if cpmf in csv_list:
      cpmf=cpmf[:-4]
      file_int=1
      while cpmf+emfil_0s(file_int)+".csv" in csv_list:
        file_int+=1
      cpmf+=emfil_0s(file_int)+".csv"
      # index有無入力
      ques=input("indexあり? y/other:")
      if ques !="" and ques[0]=="y":
        cpind=True
      else:
        cpind=False
    # データ取り出し、ファイル作成
    try:
      cpdf=pd.read_csv(cpbf)
      cpdf.to_csv(cpmf,index=cpind)
      print("info:ファイル'"+cpbf+"'のコピー'"+cpmf+"'を作成しました")
    except:
      print("エラー:ファイルコピー失敗")
    continue

  # 使用者に自力でコード入力
  elif com_list[0]=="manual":
    text=""
    while True:
      print("コード入力  'run'で実行  'exit'で終了  'see'で入力したコードを確認")
      order=input(">>")
      if order.lower()=="run":
        try:
          exec(text)
        except:
          print("エラー:実行可能なコードを入力してください")
      elif order.lower()=="exit":
        break
      elif order.lower()=="see":
        print(text)
      else:
        text+=order
    continue
  else:
    print("エラー:不適切なコマンドです")
print("info:プログラム終了")





コマンド入力>>fuaivuha\faj;vba;\ajnvj\sra\\


['fuaivuha', 'faj;vba;', 'ajnvj', 'sra']